In [11]:
# Install libraries the first time
# ! pip install -q ipynb yfinance pandas

In [12]:
import yfinance as yf
import pandas as pd

### Variables

In [13]:
timeFrame = '5y'  # Options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
symbols = [
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corporation
    'AMZN',  # Amazon.com, Inc.
    'GOOGL', # Alphabet Inc. (Class A)
    'GOOG',  # Alphabet Inc. (Class C)
    'META',  # Meta Platforms, Inc.
    'TSLA',  # Tesla, Inc.
    'NVDA',  # NVIDIA Corporation
    'PYPL',  # PayPal Holdings, Inc.
    'ADBE',  # Adobe Inc.
    'NFLX',  # Netflix, Inc.
    'INTC',  # Intel Corporation
    'CMCSA', # Comcast Corporation
    'PEP',   # PepsiCo, Inc.
    'CSCO',  # Cisco Systems, Inc.
    'AVGO',  # Broadcom Inc.
    'COST',  # Costco Wholesale Corporation
    'AMGN',  # Amgen Inc.
    'TXN',   # Texas Instruments Incorporated
    'QCOM',  # QUALCOMM Incorporated
    'HON',   # Honeywell International Inc.
    'SBUX',  # Starbucks Corporation
    'AMD',   # Advanced Micro Devices, Inc.
    'INTU',  # Intuit Inc.
    'ISRG',  # Intuitive Surgical, Inc.
    'BKNG',  # Booking Holdings Inc.
    'MDLZ',  # Mondelez International, Inc.
    'ADP',   # Automatic Data Processing, Inc.
    'GILD',  # Gilead Sciences, Inc.
    'MU',    # Micron Technology, Inc.
    'AMAT',  # Applied Materials, Inc.
    'LRCX',  # Lam Research Corporation
    'ADI',   # Analog Devices, Inc.
    'ASML',  # ASML Holding N.V.
    'MRNA',  # Moderna, Inc.
    'ZM',    # Zoom Video Communications, Inc.
    'DOCU',  # DocuSign, Inc.
    'ROST',  # Ross Stores, Inc.
    'REGN',  # Regeneron Pharmaceuticals, Inc.
    'VRTX',  # Vertex Pharmaceuticals Incorporated
    'EBAY',  # eBay Inc.
    'EXC',   # Exelon Corporation
    'SNPS',  # Synopsys, Inc.
    'WDAY',  # Workday, Inc.
    'TEAM',  # Atlassian Corporation Plc
    'OKTA',  # Okta, Inc.
    'CRWD'   # CrowdStrike Holdings, Inc.
]

### Getting historic data

In [14]:
# Define a function to calculate FutureYearChange
def calculate_future_year_change(ticker_symbol, timeframe, buffer=1):
    """
    Calculate Future Year Change for a ticker, requesting a larger valid period to ensure full data coverage.
    """
    # Map the valid period for yfinance
    valid_periods = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    extended_timeframe = 'max' if timeframe == 'max' else valid_periods[valid_periods.index(timeframe) + buffer]

    # Get historical data
    ticker = yf.Ticker(ticker_symbol)
    futureChange = ticker.history(period=extended_timeframe)
    
    # Calculate Future Year Change
    futureChange['Future Year Change'] = (futureChange['Close'].shift(-252) / futureChange['Close'] - 1)

    # Define the valid date range: 2 years ago to 1 year ago
    end_date = futureChange.index[-1] - pd.DateOffset(years=1)  # 1 year ago
    start_date = end_date - pd.DateOffset(years=int(timeframe[:-1]))  # 2 years ago
    futureChange = futureChange.loc[start_date:end_date]

    # Drop rows where Future Year Change is NaN
    futureChange = futureChange.dropna(subset=['Future Year Change'])

    return futureChange

# Data collection
histData = pd.DataFrame()
for symbol in symbols:
    try:
        futureChange = calculate_future_year_change(symbol, timeFrame)
        futureChange['Ticker'] = symbol  # Add Ticker column
        histData = pd.concat([histData, futureChange])
    except Exception as e:
        print(f"Error processing {symbol}: {e}")

# Reorder columns to have 'Ticker' right after the date
histData.reset_index(inplace=True)  # Move the date index to a column
columns_order = ['Date', 'Ticker'] + [col for col in histData.columns if col not in ['Date', 'Ticker']]
histData = histData[columns_order]

# Save to CSV
histData.to_csv('data.csv', index=False)

In [15]:
def get_static_ev_data(ticker_symbol):
    """
    Fetch static data needed for EV calculation: Total Debt, Cash, and Shares Outstanding.
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info

        # Retrieve static components
        total_debt = info.get('totalDebt', 0)  # Defaults to 0 if not available
        cash = info.get('totalCash', 0)       # Defaults to 0 if not available
        shares_outstanding = info.get('sharesOutstanding', None)  # None if not available

        print(f"Static Data for {ticker_symbol}:")
        print(f"  Total Debt: {total_debt}")
        print(f"  Cash: {cash}")
        print(f"  Shares Outstanding: {shares_outstanding}")

        return total_debt, cash, shares_outstanding
    except Exception as e:
        print(f"Error fetching EV components for {ticker_symbol}: {e}")
        return None, None, None


In [16]:
def calculate_approx_ebit(ticker_symbol):
    """
    Approximate EBIT using Revenue and Operating Income (if available).
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info

        # Try fetching revenue and operating income
        revenue = info.get('totalRevenue', None)
        operating_income = info.get('operatingIncome', None)

        if revenue and operating_income:
            return operating_income  # Use direct value if available
        elif revenue:
            # If operating income is unavailable, assume a fixed margin
            estimated_margin = 0.15  # Example: 15% operating margin
            return revenue * estimated_margin
        else:
            return None
    except Exception as e:
        print(f"Error calculating EBIT for {ticker_symbol}: {e}")
        return None


In [17]:
def fetch_ebit(ticker_symbol):
    """
    Fetch or approximate EBIT for the ticker symbol.
    """
    ebit = calculate_approx_ebit(ticker_symbol)
    if ebit is not None:
        print(f"Approximated EBIT for {ticker_symbol}: {ebit}")
    else:
        print(f"EBIT for {ticker_symbol} is unavailable.")
    return ebit


In [18]:
def calculate_daily_ev_ebit_for_histdata(hist_data):
    """
    Add a column for daily EV/EBIT to existing historical data.
    """
    data_with_ev_ebit = hist_data.copy()
    data_with_ev_ebit['EV/EBIT'] = None  # Initialize the column

    for symbol in data_with_ev_ebit['Ticker'].unique():
        # Filter data for the current ticker
        ticker_data = data_with_ev_ebit[data_with_ev_ebit['Ticker'] == symbol].copy()

        # Fetch static EV data
        total_debt, cash, shares_outstanding = get_static_ev_data(symbol)

        if shares_outstanding is None:
            print(f"Missing shares outstanding for {symbol}. Skipping EV/EBIT calculation.")
            continue

        # Fetch EBIT
        ebit = fetch_ebit(symbol)

        if ebit is None or ebit == 0:
            print(f"Missing or invalid EBIT for {symbol}. Skipping EV/EBIT calculation.")
            continue

        # Calculate EV and EV/EBIT
        ticker_data['EV'] = (ticker_data['Close'] * shares_outstanding) + total_debt - cash
        ticker_data['EV/EBIT'] = ticker_data['EV'] / ebit

        # Update the main DataFrame
        data_with_ev_ebit.loc[ticker_data.index, 'EV/EBIT'] = ticker_data['EV/EBIT']

    # Drop intermediate columns if present
    if 'EV' in data_with_ev_ebit.columns:
        data_with_ev_ebit.drop(columns=['EV'], inplace=True)

    return data_with_ev_ebit


In [19]:
# Function to calculate ROIC
def calculate_daily_roic_for_histdata(hist_data):
    """
    Add daily ROIC (Return on Invested Capital) to existing historical data.
    """
    data_with_roic = hist_data.copy()

    for symbol in data_with_roic['Ticker'].unique():
        ticker_data = data_with_roic[data_with_roic['Ticker'] == symbol].copy()

        # Fetch static EV data
        total_debt, cash, shares_outstanding = get_static_ev_data(symbol)

        if shares_outstanding:
            # Calculate Market Cap and Invested Capital
            ticker_data['Market Cap'] = ticker_data['Close'] * shares_outstanding
            ticker_data['Invested Capital'] = total_debt + ticker_data['Market Cap'] - cash
        else:
            ticker_data['Invested Capital'] = None

        # Fetch EBIT and approximate NOPAT
        ebit = fetch_ebit(symbol)
        if ebit:
            # Assume a tax rate of 21% (adjust as needed)
            tax_rate = 0.21
            nopat = ebit * (1 - tax_rate)
        else:
            nopat = None

        if nopat and ticker_data['Invested Capital'].notnull().all():
            # Calculate ROIC
            ticker_data['ROIC'] = nopat / ticker_data['Invested Capital']
        else:
            ticker_data['ROIC'] = None

        # Update data with calculated ROIC
        data_with_roic.loc[ticker_data.index, 'Market Cap'] = ticker_data['Market Cap']
        data_with_roic.loc[ticker_data.index, 'ROIC'] = ticker_data['ROIC']

    return data_with_roic

In [20]:
histData = calculate_daily_ev_ebit_for_histdata(histData)
histData = calculate_daily_roic_for_histdata(histData)

# Save updated data to a CSV file
histData.to_csv('data.csv', index=False)

Static Data for AAPL:
  Total Debt: 119058997248
  Cash: 65171001344
  Shares Outstanding: 15037899776
Approximated EBIT for AAPL: 58655249203.2
Static Data for MSFT:
  Total Debt: 96838000640
  Cash: 78429003776
  Shares Outstanding: 7434880000
Approximated EBIT for MSFT: 38128499097.6
Static Data for AMZN:
  Total Debt: 158534991872
  Cash: 88050999296
  Shares Outstanding: 10515000320
Approximated EBIT for AMZN: 93019196620.8
Static Data for GOOGL:
  Total Debt: 29289000960
  Cash: 93229998080
  Shares Outstanding: 5842999808
Approximated EBIT for GOOGL: 50978852044.799995
Static Data for GOOG:
  Total Debt: 29289000960
  Cash: 93229998080
  Shares Outstanding: 5534000128
Approximated EBIT for GOOG: 50978852044.799995
Static Data for META:
  Total Debt: 49046999040
  Cash: 70899998720
  Shares Outstanding: 2180000000
Approximated EBIT for META: 23434049126.399998
Static Data for TSLA:
  Total Debt: 12782999552
  Cash: 33648001024
  Shares Outstanding: 3210060032
Approximated EBIT fo

In [21]:
def getTickerData(ticker_symbol):
    # Fetch historical data for the ticker
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(period='1d')
    
    # Fetch static EV data
    total_debt, cash, shares_outstanding = get_static_ev_data(ticker_symbol)
    
    # Fetch EBIT
    ebit = fetch_ebit(ticker_symbol)
    
    # Calculate EV and EV/EBIT
    ev = (hist['Close'].iloc[-1] * shares_outstanding) + total_debt - cash
    ev_ebit = ev / ebit if ebit else None
    
    # Calculate Market Cap
    market_cap = hist['Close'].iloc[-1] * shares_outstanding
    
    # Calculate ROIC
    tax_rate = 0.21  # Example tax rate
    nopat = ebit * (1 - tax_rate) if ebit else None
    invested_capital = total_debt + market_cap - cash
    roic = nopat / invested_capital if nopat and invested_capital else None
    
    # Prepare the result
    result = {
        'Open': hist['Open'].iloc[-1],
        'High': hist['High'].iloc[-1],
        'Low': hist['Low'].iloc[-1],
        'Close': hist['Close'].iloc[-1],
        'Volume': hist['Volume'].iloc[-1],
        'Dividends': hist['Dividends'].iloc[-1] if 'Dividends' in hist.columns else 0.0,
        'Stock Splits': hist['Stock Splits'].iloc[-1] if 'Stock Splits' in hist.columns else 0.0,
        'EV/EBIT': ev_ebit,
        'Market Cap': market_cap,
        'ROIC': roic
    }
    
    return result